# TSTR CTGAN Dataset F

In [2]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os
print('Libraries imported!!')

Libraries imported!!


In [3]:
#define directory of functions and actual directory
HOME_PATH = '' #home path of the project
FUNCTIONS_DIR = "EVALUATION FUNCTIONS/UTILITY"
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(HOME_PATH + FUNCTIONS_DIR)

#import functions for data labelling analisys
from utility_evaluation import DataPreProcessor
from utility_evaluation import train_evaluate_model

#change directory to actual directory
os.chdir(ACTUAL_DIR)
print('Functions imported!!')

Functions imported!!


## 1. Read data

In [4]:
#read real dataset
train_data = pd.read_csv(HOME_PATH + 'SYNTHETIC DATASETS/CTGAN/F_IndianLiverPatient_Synthetic_CTGAN.csv')
categorical_columns = ['gender','class']
for col in categorical_columns :
    train_data[col] = train_data[col].astype('category')
train_data

,age,gender,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G,class
0,48,Male,0.4,0.3,293.3,90.9,122.1,6.9,4.8,373.75,2.0
1,63,Male,5.1,1.4,146.7,44.4,767.0,4.5,2.6,215.70,2.0
2,46,Male,2.6,0.2,279.5,8.1,71.5,3.3,3.8,105.82,1.0
3,32,Male,4.9,0.5,109.0,6.9,25.9,5.2,1.3,417.34,2.0
4,54,Male,2.2,0.3,118.1,28.7,56.4,7.2,3.8,29.17,2.0
...,...,...,...,...,...,...,...,...,...,...,...
461,4,Female,0.9,0.4,218.1,19.3,7.3,8.8,2.5,91.13,2.0
462,59,Male,1.1,1.7,94.3,3.2,3.0,6.3,3.0,453.24,2.0
463,68,Female,2.0,0.7,250.9,47.8,76.6,6.0,4.6,331.60,1.0
464,66,Male,1.8,1.6,113.4,4.3,843.0,7.9,4.9,467.79,1.0


In [6]:
#read test data
test_data = pd.read_csv(HOME_PATH + 'REAL DATASETS/TEST DATASETS/F_IndianLiverPatient_Real_Test.csv')
for col in categorical_columns :
    test_data[col] = test_data[col].astype('category')
test_data

,age,gender,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G,class
0,19,Male,1.4,0.8,178.0,13.0,26.0,8.0,4.6,1.30,2.0
1,12,Male,1.0,0.2,719.0,157.0,108.0,7.2,3.7,1.00,1.0
2,60,Male,5.7,2.8,214.0,412.0,850.0,7.3,3.2,0.78,1.0
3,42,Female,0.5,0.1,162.0,155.0,108.0,8.1,4.0,0.90,1.0
4,40,Male,14.5,6.4,358.0,50.0,75.0,5.7,2.1,0.50,1.0
...,...,...,...,...,...,...,...,...,...,...,...
112,60,Male,0.9,0.3,168.0,16.0,24.0,6.7,3.0,0.80,1.0
113,54,Female,1.4,0.7,195.0,36.0,16.0,7.9,3.7,0.90,2.0
114,47,Female,3.0,1.5,292.0,64.0,67.0,5.6,1.8,0.47,1.0
115,27,Male,1.3,0.6,106.0,25.0,54.0,8.5,4.8,-100000.00,2.0


In [7]:
target = 'class'
#quick look at the breakdown of class values
print('Train data')
print(train_data.shape)
print(train_data.groupby(target).size())
print('#####################################')
print('Test data')
print(test_data.shape)
print(test_data.groupby(target).size())

Train data
(466, 11)
class
1.0    278
2.0    188
dtype: int64
#####################################
Test data
(117, 11)
class
1.0    87
2.0    30
dtype: int64


## 2. Pre-process training data

In [8]:
target = 'class'
categorical_columns = ['gender']
numerical_columns = train_data.select_dtypes(include=['int64','float64']).columns.tolist()
categories = [np.array([0, 1])]

data_preprocessor = DataPreProcessor(categorical_columns, numerical_columns, categories)
x_train = data_preprocessor.preprocess_train_data(train_data.loc[:, train_data.columns != target])
y_train = train_data.loc[:, target]

x_train.shape, y_train.shape

((466, 11), (466,))

## 3. Preprocess test data

In [9]:
x_test = data_preprocessor.preprocess_test_data(test_data.loc[:, test_data.columns != target])
y_test = test_data.loc[:, target]
x_test.shape, y_test.shape

((117, 11), (117,))

## 4. Create a dataset to save the results

In [10]:
results = pd.DataFrame(columns = ['model','accuracy','precision','recall','f1'])
results

,model,accuracy,precision,recall,f1


## 4. Train and evaluate Random Forest Classifier

In [11]:
rf_results = train_evaluate_model('RF', x_train, y_train, x_test, y_test)
results = results.append(rf_results, ignore_index=True)
rf_results

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished


,model,accuracy,precision,recall,f1
0,RF,0.6325,0.6519,0.6325,0.6412


## 5. Train and Evaluate KNeighbors Classifier

In [12]:
knn_results = train_evaluate_model('KNN', x_train, y_train, x_test, y_test)
results = results.append(knn_results, ignore_index=True)
knn_results

,model,accuracy,precision,recall,f1
0,KNN,0.6838,0.6941,0.6838,0.6885


## 6. Train and evaluate Decision Tree Classifier

In [13]:
dt_results = train_evaluate_model('DT', x_train, y_train, x_test, y_test)
results = results.append(dt_results, ignore_index=True)
dt_results

,model,accuracy,precision,recall,f1
0,DT,0.5726,0.5411,0.5726,0.556


## 7. Train and evaluate Support Vector Machines Classifier

In [14]:
svm_results = train_evaluate_model('SVM', x_train, y_train, x_test, y_test)
results = results.append(svm_results, ignore_index=True)
svm_results

[LibSVM]

,model,accuracy,precision,recall,f1
0,SVM,0.453,0.7683,0.453,0.4458


## 8. Train and evaluate Multilayer Perceptron Classifier

In [15]:
mlp_results = train_evaluate_model('MLP', x_train, y_train, x_test, y_test)
results = results.append(mlp_results, ignore_index=True)
mlp_results

Iteration 1, loss = 0.69149372
Iteration 2, loss = 0.67281482
Iteration 3, loss = 0.66430956
Iteration 4, loss = 0.65989612
Iteration 5, loss = 0.65411082
Iteration 6, loss = 0.64617800
Iteration 7, loss = 0.64014738
Iteration 8, loss = 0.63458387
Iteration 9, loss = 0.62965315
Iteration 10, loss = 0.62431165
Iteration 11, loss = 0.61875180
Iteration 12, loss = 0.61383538
Iteration 13, loss = 0.60861052
Iteration 14, loss = 0.60375299
Iteration 15, loss = 0.59937835
Iteration 16, loss = 0.59414189
Iteration 17, loss = 0.58882228
Iteration 18, loss = 0.58486911
Iteration 19, loss = 0.58033279
Iteration 20, loss = 0.57543297
Iteration 21, loss = 0.56879647
Iteration 22, loss = 0.56760997
Iteration 23, loss = 0.56161130
Iteration 24, loss = 0.55502053
Iteration 25, loss = 0.54967100
Iteration 26, loss = 0.54356453
Iteration 27, loss = 0.53850268
Iteration 28, loss = 0.53417865
Iteration 29, loss = 0.52808338
Iteration 30, loss = 0.52194400
Iteration 31, loss = 0.51684730
Iteration 32, los

,model,accuracy,precision,recall,f1
0,MLP,0.5214,0.723,0.5214,0.5416


## 9. Save results file

In [16]:
results.to_csv('RESULTS/models_results_ctgan.csv', index=False)
results

,model,accuracy,precision,recall,f1
0,RF,0.6325,0.6519,0.6325,0.6412
1,KNN,0.6838,0.6941,0.6838,0.6885
2,DT,0.5726,0.5411,0.5726,0.5560
3,SVM,0.4530,0.7683,0.4530,0.4458
4,MLP,0.5214,0.7230,0.5214,0.5416
